In [14]:
import numpy as np
import pandas as pd
import os

In [32]:
data_directory = "../Data/Study/B/23/"
subdirectory = "recalibrated_calibration/"
fileList = os.listdir(data_directory + subdirectory)

In [16]:
def clean_df(df):
    result = df.loc[(df['Movement'] != "start") & (df['Movement'] != "transition") & (df['EyeTrackingEnabled'] == True) & (df['EyeCalibrationValid'] == True) & (df['EyeTrackingEnabledAndValid'] == True) & (df['EyeTrackingDataValid'] == True) & (df['GazeInputSource'] == "Eyes")].copy()
    #result = result[result['Movement'] != "transition"]
    #result = result[result['EyeTrackingDataValid'] == 'TRUE']
    #result = result[result['GazeInputSource'] == 'Eyes']
    result.reset_index(inplace=True, drop=True)
    return result

In [17]:
def calculate_cosine_error(df):
    df['cosineError'] = df.apply(lambda row: np.degrees(np.arccos(np.dot(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z'])), np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))/(np.linalg.norm(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z']))) * np.linalg.norm(np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))))), axis=1)
    return df

In [18]:
def calculate_euclidean_error(df):
    #df['euclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['GazeAngleX'], row['GazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    #df['recalibratedEuclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['CalibratedGazeAngleX'], row['CalibratedGazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    df['euclideanError'] = np.sqrt((df.GazeAngleX-df.TargetAngleX) * (df.GazeAngleX-df.TargetAngleX) + (df.GazeAngleY-df.TargetAngleY) * (df.GazeAngleY-df.TargetAngleY))
    df['recalibratedEuclideanError'] = np.sqrt((df.CalibratedGazeAngleX-df.TargetAngleX) * (df.CalibratedGazeAngleX-df.TargetAngleX) + (df.CalibratedGazeAngleY-df.TargetAngleY) * (df.CalibratedGazeAngleY-df.TargetAngleY))
    return df

In [20]:
def calculate_errors(output_subdirectory):
    for file in fileList:
        if '.csv' in file:
            df = pd.read_csv(data_directory + subdirectory + file)
            error_df = calculate_euclidean_error(df)
            error_df = calculate_cosine_error(error_df)
            if not os.path.exists(os.path.join(data_directory, output_subdirectory)):
                os.makedirs(os.path.join(data_directory, output_subdirectory))
            error_df_output_path = data_directory + output_subdirectory + '/' + file
            error_df.to_csv(error_df_output_path, index=False)
            print(file)
            print("uncalibrated: " + str(error_df['euclideanError'].mean()))
            print("calibrated: " + str(error_df['recalibratedEuclideanError'].mean()))
            print("cosine: " + str(error_df['cosineError'].mean()))
            print("")

In [33]:
calculate_errors("error/recalibrated_calibration")

ssHeadConstrained_20230515091527_shifted26.csv
uncalibrated: 9.630241616831748
calibrated: 4.544948818148545
cosine: 9.434579805823724

hallway_20230515092642_shifted38.csv
uncalibrated: 25.068548146274615
calibrated: 21.23533694380781
cosine: 24.426812426844375

wsWalking_20230515093205_shifted13.csv
uncalibrated: 57.7484189213253
calibrated: 51.43620649681045
cosine: 81.70558098104429

calibration_20230515093412_shifted0.csv
uncalibrated: 4.488962088995188
calibrated: 3.3161491485869004
cosine: 4.393259224591589

calibration_20230515092025_shifted0.csv
uncalibrated: 11.30371799315887
calibrated: 4.463382204462785
cosine: 11.067286659327847

ssWalking_20230515090856_shifted33.csv
uncalibrated: 57.531744866345015
calibrated: 53.598981274062865
cosine: 87.87617338459603

wsBodyConstrained_20230515090551_shifted47.csv
uncalibrated: 24.741405153242944
calibrated: 22.86028520429386
cosine: 23.907050837616637

wsBodyConstrained_20230515092151_shifted47.csv
uncalibrated: 24.991522737176627
c